# Modelagem: montagem de elenco para times de futebol

## Importando bibliotecas e carregando dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
player_data = pd.read_csv("player-data-full.csv")

/var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/ipykernel_69241/484434192.py:1: DtypeWarning: Columns (15,16,34,35,36,37,39,73) have mixed types. Specify dtype option on import or set low_memory=False.
  player_data = pd.read_csv("player-data-full.csv")


In [3]:
player_data.head(3)

,player_id,version,name,full_name,description,image,height_cm,weight_kg,dob,positions,...,composure,defensive_awareness,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,play_styles
0,239085,240033,Erling Haaland,Erling Braut Haaland,"Erling Haaland (Erling Braut Haaland, born 21 ...",https://cdn.sofifa.net/players/239/085/24_120.png,195,94,2000-07-21,ST,...,87,38,47,29,7,14,13,11,7.0,"Acrobatic +,Power Header,Quick Step"
1,231747,240033,Kylian Mbappé,Kylian Mbappé Lottin,"Kylian Mbappé (Kylian Mbappé Lottin, born 20 D...",https://cdn.sofifa.net/players/231/747/24_120.png,182,75,1998-12-20,"ST,LW",...,88,26,34,32,13,5,7,11,6.0,"Quick Step +,Finesse Shot,Rapid,Flair,Trivela,..."
2,192985,240033,Kevin De Bruyne,Kevin De Bruyne,Kevin De Bruyne (born 28 June 1991) is a Belgi...,https://cdn.sofifa.net/players/192/985/24_120.png,181,75,1991-06-28,"CM,CAM",...,88,66,70,53,15,13,5,10,13.0,"Incisive Pass +,Dead Ball,Pinged Pass,Long Bal..."


In [4]:
player_data.columns

Index(['player_id', 'version', 'name', 'full_name', 'description', 'image',
       'height_cm', 'weight_kg', 'dob', 'positions', 'overall_rating',
       'potential', 'value', 'wage', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate', 'body_type',
       'real_face', 'release_clause', 'specialities', 'club_id', 'club_name',
       'club_league_id', 'club_league_name', 'club_logo', 'club_rating',
       'club_position', 'club_kit_number', 'club_joined',
       'club_contract_valid_until', 'country_id', 'country_name',
       'country_league_id', 'country_league_name', 'country_flag',
       'country_rating', 'country_position', 'country_kit_number', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'fk_accuracy', 'long_passing', 'ball_control',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
 

## Filtrando as colunas importantes

In [5]:
cols = ["name", "dob", "country_name", "positions", "overall_rating", "potential", "value", "wage",
        "club_name", "club_league_name", "country_name", "acceleration", "agility", "strength", "stamina"]

df = player_data[cols].copy()

df.head(1)

,name,dob,country_name,positions,overall_rating,potential,value,wage,club_name,club_league_name,country_name,acceleration,agility,strength,stamina
0,Erling Haaland,2000-07-21,Norway,ST,91,94,€185M,€340K,Manchester City,Premier League,Norway,82,78,93,76


Agora, seria interessante fazer alguns tratamentos:

- transformar a data de nascimento (`dob`) em idade

- transformar `value` e `wage` para valores numéricos

- usar as 4 colunas de informações físicas para gerar apenas um valor: `physical`

In [6]:
# --- 1. Converter 'value' e 'wage' para Números ---

def converter_valor_monetario(valor_str):
    """
    Função para converter strings como '€10.5M' ou '€200K' para um número float.
    Ex: '€10.5M' -> 10500000.0
    """
    # Verifica se o valor é uma string antes de processar
    if isinstance(valor_str, str):
        valor_str = valor_str.replace('€', '').strip()
        if 'M' in valor_str:
            return float(valor_str.replace('M', '')) * 1000000
        elif 'K' in valor_str:
            return float(valor_str.replace('K', '')) * 1000
    # Retorna o valor como está se não for uma string (pode ser NaN ou um número)
    return float(valor_str)

# Aplica a função nas colunas 'value' e 'wage' para criar colunas numéricas
df['value_eur'] = df['value'].apply(converter_valor_monetario)
df['wage_eur'] = df['wage'].apply(converter_valor_monetario)


# --- 2. Calcular a Idade a partir da Data de Nascimento ('dob') ---

# Converte a coluna 'dob' para o formato de data
df['dob'] = pd.to_datetime(df['dob'])

# Define o ano atual (considerando a data de hoje)
ano_atual = pd.to_datetime('now').year

# Calcula a idade
df['age'] = ano_atual - df['dob'].dt.year

# --- 3. Criar a Métrica de 'Fisico' ---

# O físico vai ser a média de 4 stats físicos de um jogador 

df['physical'] = ((df['stamina']  +
                df['strength']  +
                df['acceleration']  +
                df['agility'] ) / 4).round(0)

# Cria a coluna 'growth_potential' (Potencial - Overall)
df['growth_potential'] = df['potential'] - df['overall_rating']




# --- 4. Processar 'positions' para Posição Primária e Versatilidade ---

df['main_position'] = df['positions'].str.split(',').str[0].str.strip()

df['sec_positions'] = df['positions'].str.split(',').str[1:].str.join(', ').str.strip()

# Calcula a 'versatilidade' contando o número de posições que um jogador pode atuar
df['versatility'] = df['positions'].str.split(',').str.len()


cols = ["name", "age", "country_name", "main_position", "sec_positions", 
        "overall_rating", "potential", "growth_potential", 
        "value", "wage", "club_name", "club_league_name", 
        "physical", "versatility", "value_eur", "wage_eur"]

df = df[cols]


In [7]:
df.head()

,name,age,country_name,country_name,main_position,sec_positions,overall_rating,potential,growth_potential,value,wage,club_name,club_league_name,physical,versatility,value_eur,wage_eur
0,Erling Haaland,25,Norway,Norway,ST,,91,94,3,€185M,€340K,Manchester City,Premier League,82.0,1,185000000.0,340000.0
1,Kylian Mbappé,27,France,France,ST,LW,91,94,3,€181.5M,€230K,Paris Saint Germain,Ligue 1,89.0,2,181500000.0,230000.0
2,Kevin De Bruyne,34,Belgium,Belgium,CM,CAM,91,91,0,€103M,€350K,Manchester City,Premier League,78.0,2,103000000.0,350000.0
3,Rodri,29,Spain,Spain,CDM,CM,90,91,1,€122.5M,€260K,Manchester City,Premier League,74.0,2,122500000.0,260000.0
4,Harry Kane,32,England,England,ST,,90,90,0,€119.5M,€170K,FC Bayern München,Bundesliga,74.0,1,119500000.0,170000.0


Antes de prosseguir, vamos inserir os jogadores no Flamengo.

In [8]:
# Samuel Lino, Saúl, Jorginho, Emerson Royal, Danilo (Juventus), Michael, Alex Sandro (Juventus)

print(f"Número de jogadores antes da limpeza: {len(df)}")
df = df[df['club_name'] != 'Flamengo'].copy()
print(f"Número de jogadores após a limpeza: {len(df)}")

# --- 2. ADICIONAR NOVOS JOGADORES AO FLAMENGO ---

# Lista de dicionários para organizar os jogadores e seus identificadores únicos.
# Para nomes repetidos, especificamos o 'club_name' original.
jogadores_para_mudar = [
    {'name': 'Samuel Lino', 'club_name': None},
    {'name': 'Saúl', 'club_name': "Atlético Madrid"},
    {'name': 'Jorginho', 'club_name': None},
    {'name': 'Emerson Royal', 'club_name': None},
    {'name': 'Michael', 'club_name': None},
    {'name': 'Danilo', 'club_name': 'Juventus'},
    {'name': 'Alex Sandro', 'club_name': 'Juventus'}
]

NOVO_TIME = 'Flamengo'
NOVA_LIGA = 'Premier League' 

print(f"\nTransferindo jogadores para o {NOVO_TIME}...")

# Loop para Atualizar o DataFrame
for jogador in jogadores_para_mudar:
    nome_jogador = jogador['name']
    clube_original = jogador['club_name']
    
    # Cria a condição de filtro
    if clube_original:
        condicao = (df['name'] == nome_jogador) & (df['club_name'] == clube_original)
    else:
        condicao = (df['name'] == nome_jogador)
        
    # Usa .loc para encontrar as linhas que atendem à condição e alterar as colunas
    df.loc[condicao, ['club_name', 'club_league_name']] = [NOVO_TIME, NOVA_LIGA]


# --- 3. VERIFICAÇÃO FINAL ---

print("\n--- Verificação do Novo Elenco do Flamengo ---")

# Filtra e exibe todos os jogadores que agora pertencem ao Flamengo
# O resultado deve ser apenas os 7 jogadores que você adicionou
df[df['club_name'] == "Flamengo"]


Número de jogadores antes da limpeza: 18331
Número de jogadores após a limpeza: 18311

Transferindo jogadores para o Flamengo...

--- Verificação do Novo Elenco do Flamengo ---


,name,age,country_name,country_name,main_position,sec_positions,overall_rating,potential,growth_potential,value,wage,club_name,club_league_name,physical,versatility,value_eur,wage_eur
177,Jorginho,34,Italy,Italy,CDM,CM,83,83,0,€25.5M,€125K,Flamengo,Premier League,72.0,2,25500000.0,125000.0
338,Saúl,31,NaN,NaN,CM,,81,81,0,€25.5M,€65K,Flamengo,Premier League,72.0,1,25500000.0,65000.0
356,Danilo,34,NaN,NaN,CB,RB,81,81,0,€17.5M,€100K,Flamengo,Premier League,72.0,2,17500000.0,100000.0
511,Samuel Lino,26,NaN,NaN,LM,LWB,79,84,5,€26.5M,€48K,Flamengo,Premier League,78.0,2,26500000.0,48000.0
902,Emerson Royal,26,NaN,NaN,RB,CB,77,79,2,€12.5M,€56K,Flamengo,Premier League,74.0,2,12500000.0,56000.0
1095,Alex Sandro,34,NaN,NaN,CB,LB,77,77,0,€7M,€77K,Flamengo,Premier League,73.0,2,7000000.0,77000.0
1203,Michael,29,NaN,NaN,RM,LM,76,76,0,€8M,€36K,Flamengo,Premier League,78.0,2,8000000.0,36000.0


Esses serão os jogadores que serão mantidos na construção do novo elenco.

# Início do experimento


In [9]:
TIME_ESCOLHIDO = "Flamengo"
TRANSFER_BUDGET = 100000000  # 200 Milhões de Euros
WAGE_BUDGET_YEAR = 20000000
NUM_CONTRATACOES = 23


# Crie o mercado e o elenco atual
mercado = df[df['club_name'] != TIME_ESCOLHIDO].copy()
elenco_atual = df[df['club_name'] == TIME_ESCOLHIDO].copy()

# AQUI ESTÁ A MUDANÇA: Calcule o tamanho final em vez de defini-lo manualmente
TAMANHO_ELENCO_FINAL = len(elenco_atual) + NUM_CONTRATACOES
print(f"O elenco atual do {TIME_ESCOLHIDO} tem {len(elenco_atual)} jogadores.")
print(f"Após contratar {NUM_CONTRATACOES} jogadores, o elenco final terá {TAMANHO_ELENCO_FINAL} jogadores.")

# Requisitos mínimos de posições para o elenco final
requisitos_posicao = {
    'GK': 3,
    'CB': 4,
    'LB': 2,
    'RB': 2,
    'CDM': 3,
    'CM': 4,
    'LW': 2,
    'RW': 2,
    'ST': 3
}

# Definição dos mapas de posição (para agregar 'CM', 'CDM', 'CAM', etc.)
formacao_titular = {
    'GK': 1,
    'CB': 2,
    'LFB': 1,  # Full-back direito
    'RFB': 1,  # Full-back direito
    'MC': 3,  # Meio-Campista (Geral)
    'LWG': 1,  # Winger esquerdo
    'RWG': 1,
    'ST': 1
}

mapa_posicoes = { # Essa função faz um mapeamento de posições para a montagem da escalação
    'GK': 'GK',
    'CB': 'CB',
    'LB': 'LFB',
    'RB': 'RFB',
    'LWB': 'LFB',
    'RWB': 'RFB',
    'CDM': 'MC',
    'CM': 'MC',
    'CAM': 'MC',
    'LW': 'LWG',
    'RW': 'RWG',
    'LM': 'LWG',
    'RM': 'RWG',
    'ST': 'ST',
    'CF': 'ST'
}

NUM_TITULARES = 11

# É importante diferenciar titulares de reservas tanto para a química, quanto para garantir que o modelo
# monte um elenco equilibrado. Sem isso, ele pode preferir contratar 3 goleiros top-class e negligenciar
# o resto do time, por exemplo.

O elenco atual do Flamengo tem 7 jogadores.
Após contratar 23 jogadores, o elenco final terá 30 jogadores.


In [10]:
# Dicionários de dados separados
elenco_data = elenco_atual.to_dict('index')
players_data = mercado.to_dict('index') # 'players_data' é só o mercado

# IDs
elenco_ids = elenco_atual.index.tolist()
player_ids = mercado.index.tolist() # IDs do mercado
elenco_final_ids = elenco_ids + player_ids # IDs de todos os jogadores (atuais + mercado)

# Dicionário de dados COMPLETO (necessário para 'titular_vars')
players_data_completo = {}
players_data_completo.update(elenco_data)
players_data_completo.update(players_data)


/var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/ipykernel_69241/2719003106.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  elenco_data = elenco_atual.to_dict('index')
/var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/ipykernel_69241/2719003106.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  players_data = mercado.to_dict('index') # 'players_data' é só o mercado


## Definição do modelo (v1 - dummy)

In [11]:
import pulp

model = pulp.LpProblem("Reconstrucao_Elenco_Titulares", pulp.LpMaximize)

# --- 3. Definição das Variáveis de Decisão ---

# 'player_vars': 1 se o jogador do MERCADO for contratado
player_vars = pulp.LpVariable.dicts("Jogador_Contratado", player_ids, cat='Binary')

# 'titular_vars': 1 se o jogador do ELENCO FINAL (atuais + contratados) for titular
titular_vars = pulp.LpVariable.dicts("Titular", elenco_final_ids, cat='Binary')

# --- 4. Definição das Restrições ---

# a) Restrição de Orçamento de Transferência
model += pulp.lpSum([
    players_data[i]['value_eur'] * player_vars[i] for i in player_ids
]) <= TRANSFER_BUDGET, "Restricao_Orcamento"

# b) Restrição de Orçamento Salarial (apenas para novos contratados)
model += pulp.lpSum([
    (players_data[i]['wage_eur'] * 52) * player_vars[i] for i in player_ids
]) <= WAGE_BUDGET_YEAR, "Restricao_Salarios"

# c) Restrição de Número de Contratações
model += pulp.lpSum([player_vars[i] for i in player_ids]) == NUM_CONTRATACOES, "Restricao_Num_Contratacoes"

# d) Restrições de Posição Mínima (Profundidade do Elenco)
# (Usando o seu loop original, que não agrega posições para profundidade)
for pos, min_req in requisitos_posicao.items():
    # Conta quantos jogadores daquela posição já estão no elenco atual
    jogadores_pos_atual = elenco_atual[elenco_atual['main_position'] == pos].shape[0]
    
    # Adiciona a restrição para os jogadores a serem contratados
    model += pulp.lpSum([
        player_vars[i] for i in player_ids if players_data[i]['main_position'] == pos
    ]) >= (min_req - jogadores_pos_atual), f"Restricao_Min_{pos}"

# e) Restrições da Formação Titular (com agregação de posições)
for pos_formacao, num_necessarios in formacao_titular.items():
    
    # Cria uma lista de todos os jogadores que pertencem a essa posição genérica
    jogadores_da_posicao = [
        i for i in elenco_final_ids 
        if mapa_posicoes.get(players_data_completo[i]['main_position']) == pos_formacao
    ]
    
    # Adiciona a restrição
    model += pulp.lpSum([titular_vars[i] for i in jogadores_da_posicao]) \
                         == num_necessarios, f'Titulares_{pos_formacao}'
    
# f) Restrição de Ligação (Titular só pode ser quem está no elenco)
for i in player_ids: # Para os jogadores do mercado
    # Um jogador do mercado SÓ pode ser titular (titular_vars[i]=1)
    # SE ele for contratado (player_vars[i]=1)
    model += titular_vars[i] <= player_vars[i], f"Ligacao_Titular_Contratado_{i}"

# (Não é necessária restrição de ligação para 'elenco_ids', pois 
# o modelo assume que todos do elenco atual são mantidos)
    
# --- 5. Definição da Função Objetivo (Ponderada) ---
# (Reformatada para vir por último, como solicitado)

# Define os pesos de importância
PESO_TITULAR = 1.0   # O valor total de um titular
PESO_RESERVA = 0.15  # O valor de um reserva

# Define os pesos dos atributos do jogador
w_qualidade = 0.5
w_potencial = 0.3
w_fisico = 0.2

# Lógica da Função Objetivo:
# Score = (Soma do valor de reserva de estar no elenco) + (Soma do bônus de titular)
# Isso evita a não-linearidade (titular * reserva)

score_reserva = {}
score_titular_bonus = {}

for i in elenco_final_ids:
    p_data = players_data_completo[i]
    
    # Calcula o score base do jogador (usando 'growth_potential')
    score_base = (p_data['overall_rating'] * w_qualidade +
                  p_data.get('growth_potential', 0) * w_potencial + # .get() para segurança
                  p_data['physical'] * w_fisico)
    
    # O valor de um jogador como reserva
    score_reserva[i] = score_base * PESO_RESERVA
    # O valor ADICIONAL que ele ganha se for titular
    score_titular_bonus[i] = score_base * (PESO_TITULAR - PESO_RESERVA)

# Valor de reserva (para os que JÁ ESTÃO no elenco - são mantidos por padrão)
obj_reservas_atuais = pulp.lpSum([score_reserva[k] for k in elenco_ids])

# Valor de reserva (para os NOVOS contratados - só conta se forem contratados)
obj_reservas_novos = pulp.lpSum([score_reserva[j] * player_vars[j] for j in player_ids])

# Bônus de titular (para TODOS do elenco final - só conta se forem titulares)
obj_bonus_titulares = pulp.lpSum([score_titular_bonus[i] * titular_vars[i] for i in elenco_final_ids])

# A função objetivo final é a soma desses três componentes
model += obj_reservas_atuais + obj_reservas_novos + obj_bonus_titulares, "Score_Ponderado_Elenco"
    

In [12]:
# --- 6. Resolver o Problema ---
model.solve()

# --- 7. Exibir os Resultados ---

print(f"Status da Solução: {pulp.LpStatus[model.status]}\n")

# Verifica se uma solução ótima foi encontrada
if pulp.LpStatus[model.status] == 'Optimal':
    
    # Listas para armazenar os resultados
    contratacoes = []
    time_titular = []
    custo_total_transfer = 0
    custo_total_salario = 0

    # Itera sobre TODOS os jogadores do elenco final (atuais + mercado)
    for i in elenco_final_ids:
        
        # 1. Verifica quem foi escalado como TITULAR
        if titular_vars[i].varValue == 1:
            jogador = players_data_completo[i]
            time_titular.append(jogador)
        
        # 2. Verifica quem foi CONTRATADO (apenas do mercado)
        if i in player_ids: # 'player_ids' é a lista de IDs do mercado
            if player_vars[i].varValue == 1:
                jogador = players_data_completo[i]
                contratacoes.append(jogador)
                
                # Soma os custos apenas dos contratados
                custo_total_transfer += jogador['value_eur']
                custo_total_salario += (jogador['wage_eur'] * 52)

    # --- Imprime os resultados formatados ---

    print("--- Time Titular (11) ---")
    # Ordena o time titular por posição (usando o mapa de posições) para melhor visualização
    mapa_ordem = {pos: idx for idx, pos in enumerate(formacao_titular.keys())}
    time_titular_ordenado = sorted(
        time_titular, 
        key=lambda x: mapa_ordem.get(mapa_posicoes.get(x['main_position']), 99)
    )
    
    for jogador in time_titular_ordenado:
        print(f"  - {jogador['main_position']:<4} | {jogador['name']:<25} | "
              f"Rating: {jogador['overall_rating']}, Potencial: {jogador['potential']}")

    print(f"\n--- Novas Contratações ({len(contratacoes)}) ---")
    for jogador in sorted(contratacoes, key=lambda x: x['overall_rating'], reverse=True):
        # Verifica se o contratado também é titular
        eh_titular = "TITULAR" if jogador in time_titular else "Reserva"
        print(f"  - {jogador['main_position']:<4} | {jogador['name']:<25} | "
              f"Rating: {jogador['overall_rating']}, Potencial: +{jogador['growth_potential']}; Valor: €{jogador['value_eur']:,.0f} | Status: {eh_titular}")
    
    print("\n--- Resumo Financeiro das Contratações ---")
    print(f"Custo Total de Transferência: €{custo_total_transfer:,.2f} (Orçamento: €{TRANSFER_BUDGET:,.2f})")
    print(f"Custo Anual de Salários:     €{custo_total_salario:,.2f} (Orçamento: €{WAGE_BUDGET_YEAR:,.2f})")

else:
    print("O modelo não encontrou uma solução ótima (Pode ser Infactível ou Ilimitado).")
    print("Tente relaxar as restrições (aumentar orçamento, diminuir nº de contratações, etc.)")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/danielbarros/projects/Otimizacao_Trab/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/1dc9095c28a546e79618b91d822b6ead-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/1dc9095c28a546e79618b91d822b6ead-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18329 COLUMNS
At line 252544 RHS
At line 270869 BOUNDS
At line 307485 ENDATA
Problem MODEL has 18324 rows, 36615 columns and 124369 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 714.295 - 0.13 seconds
Cgl0004I processed model has 18306 rows, 36579 columns (36579 integer (36579 of which binary)) and 124247 elements
Cutoff increment increased from 1e-05 to 0.004995
Cbc0038I Initial state - 7 inte

In [18]:
import numpy as np

# A função de OVR (v2) estava boa, vamos mantê-la
def calcular_mudanca_anual_ovr_suavizada(age, current_ovr, potential):
    """
    Versão SUAVIZADA da mudança de 'overall' base.
    Pico de carreira mais longo e declínio mais gentil.
    """
    
    if current_ovr >= potential:
        growth = 0
    else:
        # FASE DE CRESCIMENTO (antes dos ~30 anos)
        if age < 22:
            growth = np.random.uniform(1.5, 4) # Mais controlado (era 2-5)
        elif age < 27:
            growth = np.random.uniform(1, 3)
        elif age < 30: # Pico de carreira mais longo
            growth = np.random.uniform(0, 1) # Crescimento residual
        else:
            growth = 0

    # FASE DE DECLÍNIO (após os ~30 anos)
    if age < 30:
        decline = 0
    elif age < 33:
        decline = np.random.uniform(-1, 0)   # Declínio muito leve
    elif age < 36:
        decline = np.random.uniform(-2, -1)  # Declínio moderado (era -3 a -1)
    else:
        decline = np.random.uniform(-4, -2)  # Declínio acentuado (era -6 a -3)

    return growth + decline


def evoluir_valor_uma_janela_v3(jogador_stats_anterior, novas_stats):
    """
    Calcula o NOVO valor de forma INCREMENTAL, baseado no valor ANTERIOR.
    """
    valor_anterior = jogador_stats_anterior['value_eur']
    
    # 1. Mudança por OVR
    ovr_change = novas_stats['overall_rating'] - jogador_stats_anterior['overall_rating']
    # Cada ponto de OVR muda o valor em ~8-12%
    mult_ovr = 1.0 + (ovr_change * np.random.uniform(0.08, 0.12))
    
    # 2. Mudança por Idade (só se aplica no verão, quando a idade muda)
    mult_idade = 1.0
    age_change = novas_stats['age'] - jogador_stats_anterior['age']
    
    if age_change > 0: # Se a idade mudou (janela de verão)
        age = novas_stats['age']
        if age < 29:
            mult_idade = 1.05 # Valorização natural (+5%)
        elif age < 32:
            mult_idade = 0.93 # Depreciação leve (-7%)
        elif age < 35:
            mult_idade = 0.88 # Depreciação moderada (-12%)
        else:
            mult_idade = 0.82 # Depreciação acentuada (-18%)
            
    # 3. Bônus de Especulação (para potencial de crescimento)
    # Esse bônus é aplicado toda janela, mas diminui à medida que o jogador cresce
    mult_potencial = 1.0
    if novas_stats['growth_potential'] > 0:
        # Bônus de 1.5% por ponto de "gap"
        mult_potencial = 1.0 + (novas_stats['growth_potential'] * 0.015) 
        
    # 4. Fator Aleatório de Mercado
    fator_aleatorio = np.random.uniform(0.98, 1.02) # +/- 2%
    
    novo_valor = valor_anterior * mult_ovr * mult_idade * mult_potencial * fator_aleatorio
    
    # Floor de 1M para jogadores 80+ (como antes)
    if novas_stats['overall_rating'] > 80 and novo_valor < 1000000:
        return max(novo_valor, 1000000)
        
    return round(novo_valor, -3) # Arredonda para a milhar mais próxima


def evoluir_jogador_uma_janela_v3(jogador_stats, t):
    """
    Função principal de evolução, agora usando a lógica de valor v3.
    """
    novas_stats = jogador_stats.copy()
    
    # 1. Atualiza a Idade (só no verão, t=2, t=4...)
    # (t=0 é a primeira janela, t=1 é a segunda, t=2 é a terceira)
    if t > 0 and t % 2 == 0:
        novas_stats['age'] = jogador_stats['age'] + 1
    
    # 2. Evolui Overall e Físico (usando a mesma lógica v2)
    mudanca_anual_ovr = calcular_mudanca_anual_ovr_suavizada(
        novas_stats['age'], 
        novas_stats['overall_rating'], 
        novas_stats['potential']
    )
    mudanca_anual_fisico = 0
    if novas_stats['age'] > 29:
        mudanca_anual_fisico = np.random.uniform(-2, 0)
    
    fator_aleatorio_forma = np.random.normal(0, 0.5)
    
    mudanca_ovr_janela = (mudanca_anual_ovr / 2) + fator_aleatorio_forma
    mudanca_fisico_janela = mudanca_anual_fisico / 2
    
    gap_potencial = novas_stats['potential'] - novas_stats['overall_rating']
    if mudanca_ovr_janela > 0 and mudanca_ovr_janela > gap_potencial:
        mudanca_ovr_janela = max(0, gap_potencial)
    
    novas_stats['overall_rating'] = int(round(novas_stats['overall_rating'] + mudanca_ovr_janela))
    novas_stats['physical'] = int(round(novas_stats['physical'] + mudanca_fisico_janela))
    
    if novas_stats['overall_rating'] > novas_stats['potential']:
        novas_stats['overall_rating'] = novas_stats['potential']
        
    # 3. Recalcula 'growth_potential'
    novas_stats['growth_potential'] = novas_stats['potential'] - novas_stats['overall_rating']
    
    # 4. Evolui o Valor (usando a nova lógica v3)
    # Passamos os stats *antes* da evolução (jogador_stats) e os *depois* (novas_stats)
    novas_stats['value_eur'] = evoluir_valor_uma_janela_v3(jogador_stats, novas_stats)
    
    return novas_stats

In [42]:
# --- Jogador Jovem (Ex: Arda Güler) ---
jovem = {
    'name': 'Arda Güler', 'age': 19, 'overall_rating': 77, 'potential': 88,
    'physical': 65, 'value_eur': 25000000, 'growth_potential': 11
}

# --- Jogador Veterano (Ex: Kevin De Bruyne) ---
veterano = {
    'name': 'Kevin De Bruyne', 'age': 34, 'overall_rating': 89, 'potential': 89,
    'physical': 75, 'value_eur': 60000000, 'growth_potential': 0
}

print("--- EVOLUÇÃO (V3 - INCREMENTAL) DO JOGADOR JOVEM ---")
stats_t0 = jovem
print(f"Janela 0 (Idade {stats_t0['age']}): OVR: {stats_t0['overall_rating']}, Físico: {stats_t0['physical']}, Valor: €{stats_t0['value_eur']:,}")

stats_t1 = evoluir_jogador_uma_janela_v3(stats_t0, t=1)
print(f"Janela 1 (Idade {stats_t1['age']}): OVR: {stats_t1['overall_rating']}, Físico: {stats_t1['physical']}, Valor: €{stats_t1['value_eur']:,}")

stats_t2 = evoluir_jogador_uma_janela_v3(stats_t1, t=2)
print(f"Janela 2 (Idade {stats_t2['age']}): OVR: {stats_t2['overall_rating']}, Físico: {stats_t2['physical']}, Valor: €{stats_t2['value_eur']:,}")

stats_t3 = evoluir_jogador_uma_janela_v3(stats_t2, t=3)
print(f"Janela 3 (Idade {stats_t3['age']}): OVR: {stats_t3['overall_rating']}, Físico: {stats_t3['physical']}, Valor: €{stats_t3['value_eur']:,}")

print("\n--- EVOLUÇÃO (V3 - INCREMENTAL) DO JOGADOR VETERANO ---")
stats_t0 = veterano
print(f"Janela 0 (Idade {stats_t0['age']}): OVR: {stats_t0['overall_rating']}, Físico: {stats_t0['physical']}, Valor: €{stats_t0['value_eur']:,}")

stats_t1 = evoluir_jogador_uma_janela_v3(stats_t0, t=1)
print(f"Janela 1 (Idade {stats_t1['age']}): OVR: {stats_t1['overall_rating']}, Físico: {stats_t1['physical']}, Valor: €{stats_t1['value_eur']:,}")

stats_t2 = evoluir_jogador_uma_janela_v3(stats_t1, t=2)
print(f"Janela 2 (Idade {stats_t2['age']}): OVR: {stats_t2['overall_rating']}, Físico: {stats_t2['physical']}, Valor: €{stats_t2['value_eur']:,}")

stats_t3 = evoluir_jogador_uma_janela_v3(stats_t2, t=3)
print(f"Janela 3 (Idade {stats_t3['age']}): OVR: {stats_t3['overall_rating']}, Físico: {stats_t3['physical']}, Valor: €{stats_t3['value_eur']:,}")

--- EVOLUÇÃO (V3 - INCREMENTAL) DO JOGADOR JOVEM ---
Janela 0 (Idade 19): OVR: 77, Físico: 65, Valor: €25,000,000
Janela 1 (Idade 19): OVR: 78, Físico: 65, Valor: €32,228,000.0
Janela 2 (Idade 20): OVR: 80, Físico: 65, Valor: €45,853,000.0
Janela 3 (Idade 20): OVR: 82, Físico: 65, Valor: €59,383,000.0

--- EVOLUÇÃO (V3 - INCREMENTAL) DO JOGADOR VETERANO ---
Janela 0 (Idade 34): OVR: 89, Físico: 75, Valor: €60,000,000
Janela 1 (Idade 34): OVR: 89, Físico: 74, Valor: €60,352,000.0
Janela 2 (Idade 35): OVR: 88, Físico: 73, Valor: €44,773,000.0
Janela 3 (Idade 35): OVR: 87, Físico: 73, Valor: €42,081,000.0
